# 전처리 베이스 코드

In [15]:
import os
import numpy as np
import pandas as pd

# 1. 단일 CSV 파일 경로 설정
input_csv = 'blind_test/A_5.csv'
file_name = os.path.basename(input_csv)
file_prefix = os.path.splitext(file_name)[0]

# 3. 출력 폴더 계층 생성: preprocessing_blind_A/A_5.csv
root_output = 'preprocessing_blind_A'
subfolder = file_name
output_dir = os.path.join(root_output, subfolder)
os.makedirs(output_dir, exist_ok=True)

# 2. CSV 읽기 및 전처리
#    - Datetime 또는 colec_dt/clct_dt 컬럼 처리
#    - 중복 Datetime 제거 (첫번째 값 유지)
#    - cell_volt_* 컬럼 이름 통일
#    - 분 단위 리샘플링 및 병합
#    - Unnamed: 0 제거
#    - 3.3V 미만 NaN, 날짜별 NaN 기준 제거
#    - 선형 보간
#    - 3.6V 기준 필터링
#    - 첫/마지막 날짜 제외

df = pd.read_csv(input_csv)

# Datetime 처리
if 'Datetime' in df.columns:
    df.rename(columns={'Datetime': 'colec_dt'}, inplace=True)
elif 'colec_dt' not in df.columns:
    if 'clct_dt' in df.columns:
        df.rename(columns={'clct_dt': 'colec_dt'}, inplace=True)
    else:
        raise KeyError("CSV에 'Datetime', 'colec_dt' 또는 'clct_dt' 컬럼이 없습니다.")

# datetime 변환
df['colec_dt'] = pd.to_datetime(df['colec_dt'])

# 중복 Datetime 제거: 첫번째 값만 남김
df = df.drop_duplicates(subset=['colec_dt'], keep='first')

# cell_volt 컬럼명 통일
if not any(col.startswith('cell_volt_') for col in df.columns):
    rename_map = {col: f"cell_volt_{int(col.split('_')[-1])}" 
                  for col in df.columns 
                  if col.startswith('cel_volt_') and col.split('_')[-1].isdigit()}
    df.rename(columns=rename_map, inplace=True)

# 분 단위 타임라인 생성 및 병합
timeline = pd.date_range(start=df['colec_dt'].min(),
                         end=df['colec_dt'].max(), freq='min')
merged = pd.merge(
    pd.DataFrame({'colec_dt': timeline}),
    df, on='colec_dt', how='left'
)

# Unnamed: 0 제거
merged.drop(columns=[c for c in ['Unnamed: 0'] if c in merged.columns], inplace=True)

# cell_volt 컬럼 리스트
volt_cols = [c for c in merged.columns if c.startswith('cell_volt_')]

# 3.3V 미만 NaN 처리
merged.loc[merged['cell_volt_1'] < 3.3, volt_cols] = np.nan

# 날짜 컬럼 및 NaN 기준 제거
merged['date'] = merged['colec_dt'].dt.date
nan_thresh = 1400
drop_dates = merged.groupby('date')['cell_volt_1'].apply(lambda x: x.isna().sum())
merged = merged[~merged['date'].isin(drop_dates[drop_dates > nan_thresh].index)]
merged.reset_index(drop=True, inplace=True)

# 선형 보간
merged[volt_cols] = merged[volt_cols].interpolate()

# 3.6V 유효 기준 필터링
valid_req = 400 * len(volt_cols)
valid_dates = merged.groupby('date')[volt_cols].apply(lambda x: (x >= 3.6).sum().sum())
merged = merged[merged['date'].isin(valid_dates[valid_dates >= valid_req].index)]
merged.reset_index(drop=True, inplace=True)

# 첫/마지막 날짜 제외
dates = merged['date'].unique()
if len(dates) > 2:
    merged = merged[~merged['date'].isin([dates[0], dates[-1]])]
    merged.reset_index(drop=True, inplace=True)

# 4. NumPy 배열 저장 (float32)
unique_dates = merged['date'].unique()
for col in volt_cols:
    arr = np.stack([
        merged[merged['date'] == d][col].to_numpy(dtype=np.float32)
        for d in unique_dates
    ])  # shape: (n_dates, 1440)
    save_path = os.path.join(output_dir, f"{file_prefix}_{col}.npy")
    np.save(save_path, arr)


In [13]:
# unique_dates, volt_cols, merged가 이미 정의된 상태에서 실행

bad_records = []

# 채널별로 날짜별 행 개수 체크
for col in volt_cols:
    # 날짜별 그룹 크기 구하기
    counts = merged.groupby('date')[col].size()
    # 1440이 아닌 항목만 필터
    bad = counts[counts != 1440]
    for date, cnt in bad.items():
        bad_records.append((date, col, cnt))

# 결과 출력
if bad_records:
    print("=== 1440이 아닌 행이 있는 날짜·채널 ===")
    for date, col, cnt in bad_records:
        print(f"{date} | {col} | rows: {cnt}")
else:
    print("모든 날짜·채널이 1440행을 가집니다.")


=== 1440이 아닌 행이 있는 날짜·채널 ===
2022-07-19 | cell_volt_1 | rows: 1455
2022-08-22 | cell_volt_1 | rows: 1458
2022-07-19 | cell_volt_2 | rows: 1455
2022-08-22 | cell_volt_2 | rows: 1458
2022-07-19 | cell_volt_3 | rows: 1455
2022-08-22 | cell_volt_3 | rows: 1458
2022-07-19 | cell_volt_4 | rows: 1455
2022-08-22 | cell_volt_4 | rows: 1458
2022-07-19 | cell_volt_5 | rows: 1455
2022-08-22 | cell_volt_5 | rows: 1458
2022-07-19 | cell_volt_6 | rows: 1455
2022-08-22 | cell_volt_6 | rows: 1458
2022-07-19 | cell_volt_7 | rows: 1455
2022-08-22 | cell_volt_7 | rows: 1458
2022-07-19 | cell_volt_8 | rows: 1455
2022-08-22 | cell_volt_8 | rows: 1458
2022-07-19 | cell_volt_9 | rows: 1455
2022-08-22 | cell_volt_9 | rows: 1458
2022-07-19 | cell_volt_10 | rows: 1455
2022-08-22 | cell_volt_10 | rows: 1458
2022-07-19 | cell_volt_11 | rows: 1455
2022-08-22 | cell_volt_11 | rows: 1458
2022-07-19 | cell_volt_12 | rows: 1455
2022-08-22 | cell_volt_12 | rows: 1458
2022-07-19 | cell_volt_13 | rows: 1455
2022-08-22 | c

In [14]:
# merged, unique_dates 등이 이미 정의된 상태에서 실행

# 1) 날짜별 전체 행 수 계산
counts = merged.groupby('date').size()

# 2) 1440이 아닌 날짜만 추출
bad_dates = counts[counts != 1440].index

# 3) 해당 날짜의 데이터만 필터
bad_df = merged[merged['date'].isin(bad_dates)]

# 4) Datetime(여기서는 colec_dt) 중복 행 찾기
duplicates = bad_df[bad_df['colec_dt'].duplicated(keep=False)]

# 5) 결과 출력
if not duplicates.empty:
    print("=== 1440행이 아닌 날짜 중, Datetime이 중복된 행들 ===")
    print(duplicates)
else:
    print("중복된 Datetime 행이 없습니다.")


=== 1440행이 아닌 날짜 중, Datetime이 중복된 행들 ===
                 colec_dt  bms_no      eqp_no  bsc_fg_no  bank_no  rack_no  \
36150 2022-07-19 02:30:00     1.0  20000003.0        1.0      1.0      6.0   
36151 2022-07-19 02:30:00     1.0  20000003.0        1.0      1.0      7.0   
36152 2022-07-19 02:31:00     1.0  20000003.0        1.0      1.0      6.0   
36153 2022-07-19 02:31:00     1.0  20000003.0        1.0      1.0      7.0   
36154 2022-07-19 02:32:00     1.0  20000003.0        1.0      1.0      6.0   
...                   ...     ...         ...        ...      ...      ...   
70786 2022-08-22 03:15:00     1.0  20000003.0        1.0      1.0      7.0   
70787 2022-08-22 03:16:00     1.0  20000003.0        1.0      1.0      6.0   
70788 2022-08-22 03:16:00     1.0  20000003.0        1.0      1.0      7.0   
70789 2022-08-22 03:17:00     1.0  20000003.0        1.0      1.0      6.0   
70790 2022-08-22 03:17:00     1.0  20000003.0        1.0      1.0      7.0   

       string_no  modu

# bsc_fg_no가 2일때

In [3]:
import os
import numpy as np
import pandas as pd

# 단일 CSV 파일 경로 및 기본 설정
input_csv = 'blind_test/A_2.csv'
file_name = os.path.basename(input_csv)       # 'A_2.csv'
file_prefix = os.path.splitext(file_name)[0]  # 'A_2'
root_output = 'preprocessing_blind_A'

# 전체 데이터 로드
df = pd.read_csv(input_csv)

# bsc_fg_no별로 분리 처리
for bank_no in [1, 2]:
    df_bank = df[df['bsc_fg_no'] == bank_no].copy()
    if df_bank.empty:
        print(f"No data for bsc_fg_no={bank_no}, skipping.")
        continue

    # 3. 출력 폴더 계층: preprocessing_blind_A/A_2.csv_1, A_2.csv_2
    subfolder = f"{file_name}_{bank_no}"  # 'A_2.csv_1' 등
    output_dir = os.path.join(root_output, subfolder)
    os.makedirs(output_dir, exist_ok=True)

    # 1. 시간 컬럼 처리: Datetime 또는 colec_dt/clct_dt -> colec_dt
    if 'Datetime' in df_bank.columns:
        df_bank.rename(columns={'Datetime': 'colec_dt'}, inplace=True)
    elif 'colec_dt' not in df_bank.columns:
        if 'clct_dt' in df_bank.columns:
            df_bank.rename(columns={'clct_dt': 'colec_dt'}, inplace=True)
        else:
            raise KeyError("CSV에 'Datetime', 'colec_dt' 또는 'clct_dt' 컬럼이 없습니다.")

    # 2. 전압 컬럼 이름 정규화: cel_volt_ -> cell_volt_
    if not any(col.startswith('cell_volt_') for col in df_bank.columns):
        rename_dict = {}
        for col in df_bank.columns:
            if col.startswith('cel_volt_'):
                num = col.replace('cel_volt_', '')
                if num.isdigit():
                    rename_dict[col] = f'cell_volt_{int(num)}'
        if rename_dict:
            df_bank.rename(columns=rename_dict, inplace=True)

    # datetime 변환 및 리샘플링
    df_bank['colec_dt'] = pd.to_datetime(df_bank['colec_dt'])
    start, end = df_bank['colec_dt'].min(), df_bank['colec_dt'].max()
    full_time = pd.DataFrame({'colec_dt': pd.date_range(start, end, freq='min')})
    merged = pd.merge(full_time, df_bank, on='colec_dt', how='left')

    # 불필요 컬럼 제거
    if 'Unnamed: 0' in merged.columns:
        merged.drop(columns=['Unnamed: 0'], inplace=True)

    # 채널 목록 추출
    volt_cols = [c for c in merged.columns if c.startswith('cell_volt_')]
    if not volt_cols:
        print(f"No voltage columns found for bank {bank_no}, skipping.")
        continue

    # 5. 전압 <3.3V -> NaN
    merged.loc[merged[volt_cols[0]] < 3.3, volt_cols] = np.nan
    merged['date'] = merged['colec_dt'].dt.date

    # 날짜별 NaN 기준으로 제거
    nan_thr = 1400
    valid_days = merged.groupby('date')[volt_cols[0]].apply(lambda x: x.isna().sum() <= nan_thr)
    merged = merged[merged['date'].isin(valid_days[valid_days].index)].reset_index(drop=True)

    # 6. 선형 보간
    merged[volt_cols] = merged[volt_cols].interpolate(method='linear')

    # 3.6V 기준 필터링
    vt_thr = 3.6
    req_count = 400 * len(volt_cols)
    keep_days = merged.groupby('date')[volt_cols].apply(lambda df_day: (df_day >= vt_thr).sum().sum() >= req_count)
    merged = merged[merged['date'].isin(keep_days[keep_days].index)].reset_index(drop=True)

    # 7. 첫/마지막 날짜 제외
    dates = merged['date'].unique()
    if len(dates) <= 2:
        print(f"Not enough valid days for bank {bank_no}, skipping.")
        continue
    merged = merged[~merged['date'].isin([dates[0], dates[-1]])].reset_index(drop=True)

    # 8. NumPy 배열로 저장 (float32)
    unique_dates = merged['date'].unique()
    for col in volt_cols:
        arr2d = np.stack([
            merged[merged['date'] == d][col].to_numpy(dtype=np.float32)
            for d in unique_dates
        ])  # shape: (n_dates, 1440)
        save_path = os.path.join(output_dir, f"{file_prefix}_{col}.npy")
        np.save(save_path, arr2d)
        print(f"Saved: {save_path} | shape={arr2d.shape}, dtype={arr2d.dtype}")


Saved: preprocessing_blind_A/A_2.csv_1/A_2_cell_volt_1.npy | shape=(27, 1440), dtype=float32
Saved: preprocessing_blind_A/A_2.csv_1/A_2_cell_volt_2.npy | shape=(27, 1440), dtype=float32
Saved: preprocessing_blind_A/A_2.csv_1/A_2_cell_volt_3.npy | shape=(27, 1440), dtype=float32
Saved: preprocessing_blind_A/A_2.csv_1/A_2_cell_volt_4.npy | shape=(27, 1440), dtype=float32
Saved: preprocessing_blind_A/A_2.csv_1/A_2_cell_volt_5.npy | shape=(27, 1440), dtype=float32
Saved: preprocessing_blind_A/A_2.csv_1/A_2_cell_volt_6.npy | shape=(27, 1440), dtype=float32
Saved: preprocessing_blind_A/A_2.csv_1/A_2_cell_volt_7.npy | shape=(27, 1440), dtype=float32
Saved: preprocessing_blind_A/A_2.csv_1/A_2_cell_volt_8.npy | shape=(27, 1440), dtype=float32
Saved: preprocessing_blind_A/A_2.csv_1/A_2_cell_volt_9.npy | shape=(27, 1440), dtype=float32
Saved: preprocessing_blind_A/A_2.csv_1/A_2_cell_volt_10.npy | shape=(27, 1440), dtype=float32
Saved: preprocessing_blind_A/A_2.csv_1/A_2_cell_volt_11.npy | shape=(